In [1]:
!pip install -q translate
!pip install -q underthesea==1.3.5a3
!pip install -q underthesea[deep]
!pip install -q pyvi
!pip install -q langdetect
!pip install -q googletrans==3.1.0a0
!pip install -q peft
!pip install bitsandbytes
!pip install transformers
!pip install flash-attn
!pip install -U sentence-transformers
!pip install xformers
!pip install einops

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
jupyterlab 4.2.3 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.
jupyterlab 4.2.3 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-er

In [1]:
!pip install git+https://github.com/JaidedAI/EasyOCR.git

  Cloning https://github.com/JaidedAI/EasyOCR.git to /tmp/pip-req-build-omngnxam
  Running command git clone --filter=blob:none --quiet https://github.com/JaidedAI/EasyOCR.git /tmp/pip-req-build-omngnxam
  Resolved https://github.com/JaidedAI/EasyOCR.git to commit 3d3852d8e4ddd9e5044a6f70b561ca7186d9cbac
  Preparing metadata (setup.py) ... done


In [3]:
from transformers import DPRContextEncoder, AutoProcessor, DPRContextEncoderTokenizer, BlipModel,TrOCRProcessor, VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, BitsAndBytesConfig, BlipForConditionalGeneration
#from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sentence_transformers import SentenceTransformer


# tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base', use_fast=False)
# embedding_model = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base').to(device)

embedding_model= SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

2024-08-14 15:20:58.334303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 15:20:58.334550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 15:20:58.502184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

In [6]:
import os
import torch
#import clip
from PIL import Image
#import faiss
import numpy as np
import json
import matplotlib.pyplot as plt
import math
import googletrans
import translate
import glob
import underthesea
import sys
import time
from tqdm import tqdm
from pyvi import ViUtils, ViTokenizer
from difflib import SequenceMatcher
from langdetect import detect
from pathlib import Path
import re

In [17]:
ROOT = Path(os.getcwd()).resolve()

# Add ROOT to sys.path
sys.path.append(str(ROOT))

# Determine the working directory
if len(ROOT.parents) > 1:
    WORK_DIR = ROOT.parents[0]
else:
    WORK_DIR = ROOT  # Fallback to ROOT if it doesn't have enough parents

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model, preprocess = clip.load("ViT-B/32", device=device)

paths = f"{WORK_DIR}/input/training-aic"
des_path =  f"{WORK_DIR}/working/dicts/npy_ocr"
os.makedirs(des_path, exist_ok=True)

In [13]:
print(des_path)

/kaggle/working/dicts/npy_ocr


In [ ]:
paths = "/kaggle/input/training-aic"
des_path = "/kaggle/working/"

In [53]:
class Text_Preprocessing():
    def __init__(self, stopwords_path=f"/kaggle/input/stopword/vietnamese-stopwords-dash.txt"):
        with open(stopwords_path, 'r', encoding='utf-8') as f:  # Open in text mode for easier string handling
            self.stop_words = [line.strip() for line in f.readlines()]

    def find_substring(self, string1, string2):
        match = SequenceMatcher(None, string1, string2, autojunk=False).find_longest_match(0, len(string1), 0, len(string2))
        return string1[match.a:match.a + match.size].strip()

    def remove_stopwords(self, text):
        text = ViTokenizer.tokenize(text)
        filtered_words = [w for w in text.split() if w not in self.stop_words]
        return " ".join(filtered_words)

    def lowercasing(self, text):
        return text.lower()

    def uppercasing(self, text):
        return text.upper()

    def add_accents(self, text):
        return ViUtils.add_accents(text)

    def remove_accents(self, text):
        return ViUtils.remove_accents(text)

    def sentence_segment(self, text):
        return underthesea.sent_tokenize(text)

    def text_norm(self, text):
        return underthesea.text_normalize(text)

    def text_classify(self, text):
        return underthesea.classify(text)

    def sentiment_analysis(self, text):
        return underthesea.sentiment(text)

    def __call__(self, text):
        # Apply preprocessing steps
        text = self.lowercasing(text)
        #text = self.remove_stopwords(text)
        # Uncomment and adjust as needed
        # text = self.remove_accents(text)
        # text = self.add_accents(text)
        text = self.text_norm(text)
        return text  # Return the processed text

class Translation():
    def __init__(self, from_lang='vi', to_lang='en', mode='google'):
        # The class Translation is a wrapper for the two translation libraries, googletrans and translate.
        self.__mode = mode
        self.__from_lang = from_lang
        self.__to_lang = to_lang
        self.text_processing = Text_Preprocessing()
        if mode in 'googletrans':
            self.translator = googletrans.Translator()
        elif mode in 'translate':
            self.translator = translate.Translator(from_lang=from_lang,to_lang=to_lang)

    def preprocessing(self, text):

        return self.text_processing(text) #text.lower()

    def __call__(self, text):

        text = self.preprocessing(text)
        return self.translator.translate(text) if self.__mode in 'translate' \
                else self.translator.translate(text, dest=self.__to_lang).text



In [12]:
import easyocr
reader = easyocr.Reader(['vi','en']) # this needs to run only once to load the model into memory

# Function to perform OCR on an image and return text
def ocr_image(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text
def easyocr_image(image_path):
    texts = []
    result = reader.readtext(image_path)

    for i in result:
        if i[2] >0.3:
            print(i[2],i[1])
            texts.append(i[1])
    merged_text = "\n".join(texts)
    return merged_text



In [14]:
text = easyocr_image('/kaggle/input/training-aic/Keyframes_L31/L31_V002/025.jpg')

0.5724026217395257 Hd
0.9774045414527394 Phú Yên:
0.585883441024281 lớn dánh chìm tàu dánh cá ngay cửa biển,
0.7139053604617218 người mất tích
0.31164787461920734 Phú Yên: Triêl
0.9989473819732666 Sóng


In [49]:
Translate = Translation()
for keyframe in tqdm(os.listdir(paths)):
  path_keyframe = os.path.join(paths,keyframe)
  video_paths = sorted(glob.glob(f"{path_keyframe}/*/"))
  video_paths = ['/'.join(i.split('/')[:-1]) for i in video_paths]

  start_time = time.time()
  for vd_path in video_paths:

    re_feats = []
    keyframe_paths = glob.glob(f'{vd_path}/*.jpg')
    keyframe_paths = sorted(keyframe_paths, key=lambda x : x.split('/')[-1].replace('.jpg',''))

    for keyframe_path in tqdm(keyframe_paths):
      
      #text = ocr_image(keyframe_path)

      #//////////////////////////////////
      text = easyocr_image(keyframe_path)
        
      #//////////////////////////////////
     
      text = Translate(text)

      # Convert text to embedding vector
      #embedding = embedding_model(**tokenizer(text, return_tensors="pt", padding=True, truncation=True,max_length=512, add_special_tokens = True)).pooler_output.detach().numpy()
      embeddings = embedding_model.encode(text)
      # Append embedding to re_feats list
      re_feats.append(embeddings)

    name_npy = vd_path.split('/')[-1]

    # Construct output file path
    outfile = os.path.join(des_path, f'{name_npy}.npy')

    # Ensure the directory exists before saving
    os.makedirs(des_path, exist_ok=True)
    np.save(outfile, re_feats)
    print(f"Processed {vd_path} in {time.time() - start_time} seconds")


  0%|          | 0/241 [00:00<?, ?it/s]

FUSHP
06829843
06829843
06829843


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 1/241 [00:09<36:57,  9.24s/it]

IH
7SHD
06829847
IH
06829847
ih 06829847


...


  4%|▍         | 10/241 [01:29<34:23,  8.93s/it]

IG
IS
BP
06830822
24
ac
Nalava
IS
06830822
24
is 06830822 24


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


  0%|          | 0/2 [01:45<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
feature_shape = 512


def write_bin_file_ocr(bin_path: str, npy_path: str, method='cosine'):
    if method in 'L2':
      index = faiss.IndexFlatL2(feature_shape)
    elif method in 'cosine':
      index = faiss.IndexFlatIP(feature_shape)
    else:
      assert f"{method} not supported"
    npy_files = glob.glob(os.path.join(npy_path, "*.npy"))
    npy_files_sorted = sorted(npy_files)

    for npy_file in npy_files_sorted:
        feats = np.load(npy_file)
        print(f"Loaded {npy_file}, shape: {feats.shape}")


        # Convert to float32 and reshape to match feature_shape
        feats = feats.astype(np.float32)
        feats = feats.reshape(-1, feats.shape[-1])

        # Resize or trim feats_normalized to match feature_shape if necessary
        if feats.shape[1] != feature_shape:
            feats = feats[:, :feature_shape]

        assert feats.shape[1] == feature_shape, \
            f"Query features dimension {feats.shape[1]} do not match index dimension {feature_shape}"

        # Add to Faiss index
        index.add(feats)

    # Write the Faiss index to disk
    faiss.write_index(index, os.path.join(bin_path, f"faiss_OCR_{method}.bin"))
    print(f'Saved {os.path.join(bin_path, f"faiss_OCR_{method}.bin")}')


# write ocr
write_bin_file_ocr(bin_path=f"{WORK_DIR}/data/dicts/bin_ocr", npy_path=f"{WORK_DIR}/data/dicts/npy_ocr")

